##  This is a webscrapper built to address the imbalanced dataset problem

###  This scrapes the data from the website livemint.com 

In [1]:
from bs4 import BeautifulSoup
from googlesearch import search
import requests
import pandas as pd

In [2]:
def get_livemint_urls_from_google(query,results_to_return):
    """
    Perform a Google search with a specified query on the 'livemint.com' website and return a list of URLs.

    Args:
        query (str): The search query to use, including site restriction, e.g., "US economy site:livemint.com".
        results_to_return (int): The number of search results to retrieve.

    Returns:
        list: A list of URLs corresponding to the top search results on 'livemint.com'.
    """ 
        
        
#     query = "US economy site: livemint.com"
    search_results = search(query, num_results=results_to_return)
    search_results = list(search_results)
    return search_results

In [15]:
def scrapping_the_mint(url):
    response = requests.get(url)
    if response.status_code == 200:
# Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

#     # Find and remove unwanted 'p' elements (ads)
    try:
        unwanted_p_elements = soup.find(name=['p','strong'],string='You might also like')
        for element in unwanted_p_elements:
            element.extract()
    except:
        pass
    
    try:
        for a_tag in soup.find_all('a'):
            a_tag.extract()
    #     Find and extract relevant news articles
    except:
        pass

    

    articles = soup.find('div', class_='contentSec')  # Adjust the class name to match the HTML structure
    # print(articles)

    if articles:
    #     Extract and print the content
        article_content = articles.get_text()
    #     print(article_content)
    
    try:
        if article_content.find("Read more:") != -1:
            return article_content[0:article_content.find("Read more:")]
        
        if article_content.find("Related Premium Stories")!= -1:
            return article_content[0:(article_content.find("Related Premium Stories")-400)]
    except:
        pass
    

        
        
    

In [4]:
def minor_preprocessing(article_content):
    
#     print(type(article_content))    
    try:
        
        article_content = article_content.strip()
        article_content = article_content.replace("\n","")
        article_content = article_content.replace("    ","")
        article_content = article_content.replace("\xa0"," ")
    except:
        pass
    return article_content

In [5]:
search_results = get_livemint_urls_from_google("US economy site: livemint.com",300)

In [6]:
df_us_economic_news = pd.DataFrame()
# df_us_economic_news


In [44]:
data = {"text":[]}
for i in range(len(search_results)):
    percent = i/len(search_results)*100
    percent = round(percent,2)
    print(f"Progress   {percent}%")
    article_content = scrapping_the_mint(search_results[i])
    article_content = minor_preprocessing(article_content)
    data['text'].append(article_content)
#     data += {'text':article_content,
#        'relevance':'yes'}
#     df_us_economic_news.append(data,ignore_index=True)
    
    

Progress   0.0%
Progress   0.33%
Progress   0.66%
Progress   0.99%
Progress   1.32%
Progress   1.66%
Progress   1.99%
Progress   2.32%


KeyboardInterrupt: 

In [8]:
df_us_economic_news=df_us_economic_news.append(data['text'],ignore_index=True)

/tmp/ipykernel_2507/30750174.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_us_economic_news=df_us_economic_news.append(data['text'],ignore_index=True)


In [41]:
# save dataset
temp = {df_us_economic_news.columns[0]:'text'}
df_us_economic_news=df_us_economic_news.rename(columns=temp)
df_us_economic_news['relevance'] = "yes"
df_us_economic_news.dropna(inplace=True)
df_us_economic_news.to_csv("us_economy_df.csv",header=True,index=False)
df_us_economic_news

,text,relevance
0,JPMorgan Chase CEO Jamie Dimon warned against ...,yes
1,The US economy expanded at a 2.1 per cent annu...,yes
2,An increasing number of economists — including...,yes
3,The US economy accelerated to a 2.4% annual gr...,yes
5,Rising hopes of a soft landing for the US econ...,yes
...,...,...
148,"Last year, most US investors and central banke...",yes
150,The US economy grew at a 2.3% rate in the thir...,yes
151,Federal Reserve policymakers are about to exte...,yes
154,Washington: The US economy shrank less than ex...,yes


In [37]:
df_us_economic_news['text'][12]

'Billionaire investor Warren Buffett on Saturday signaled he has lost none of his enduring confidence in the U.S. economy and his company Berkshire Hathaway Inc . In his annual letter to Berkshire shareholders, the 92-year-old Buffett urged investors to focus on the big picture over the long term, rather than higher inflation and other factors that in 2022 dampened stock prices, though not Berkshire\'s.    Warren Buffet\'s annual letter: Key highlights   Bet on Coca-ColaIn August 1994 – yes, 1994 – Berkshire completed its seven-year purchase of the 400 million shares of Coca-Cola we now own. The total cost was $1.3 billion – then a very meaningful sum at Berkshire.The cash dividend we received from Coke in 1994 was $75 million. By 2022, the dividend had increased to $704 million. Growth occurred every year, just as certain as birthdays. All Charlie and I were required to do was cash Coke’s quarterly dividend checks. We expect that those checks are highly likely to grow.  Dividends from

In [42]:
df_us_economic_news.shape

(108, 2)